In [235]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import cloudscraper
from datetime import date
from time import sleep

In [ ]:
class HltvScraper():
    def __init__(self) -> None:
        self.scraper = cloudscraper.create_scraper(browser={"browser": "firefox", "platform": "windows"})
        self.url_base = "https://www.hltv.org"

        today = date.today()
        self.def_params(today.replace(year=2023), today, "all", "all", "all")

    def def_params(self, statDate, endDate, matchType, maps, rankingFilter):
        self.statDate = statDate
        self.endDate = endDate
        self.matchType = matchType
        self.maps = maps
        self.rankingFilter = rankingFilter
        
        self.params = f"?startDate={self.statDate}&endDate={self.endDate}&matchType={self.matchType}&maps={self.maps}&rankingFilter={self.rankingFilter}"
        
    def jugadores_team(self, team: str) -> dict:
        """
        team: /id/nombre (e.g. /4608/natus-vincere)
        """
        url_team = self.url_base + "/stats/teams/" + team + self.params
        response = self.scraper.get(url_team)

        if response.status_code != 200:
            return

        soup = bs(response.text)
        list_reset_grid = soup.find_all(class_="grid reset-grid")
        list_team_info = list_reset_grid[0].find_all(class_="teammate-info standard-box")

        jugardores = {}
        for box in list_team_info:
            link = box.find("a")["href"]
            teammate = box.find(class_="text-ellipsis").text

            jugardores[teammate] = link

        return jugardores
    
    def stats_individuales(self, jugador: str):
        """
        jugardor: /id/nombre (e.g. /9816/aleksib)
        """
        nombre = jugador.split("/")[2]
        url_team = self.url_base + "/stats/players/individual" + jugador + self.params
        response = self.scraper.get(url_team)

        if response.status_code != 200:
            return
        
        soup = bs(response.text)
        tabla = soup.find(class_="columns")
        list_standard_box = tabla.find_all(class_="standard-box")

        counter = 0
        stats = {}
        group = ["Overall stats", "Opening stats", "Round stats", "Weapon stats"]
        
        for box in list_standard_box:
            stat_group = {}
            
            for row in box.find_all(class_="stats-row"):
                labels = [span.text for span in row.find_all('span')]
                stat_group[labels[0]] = labels[-1]

            stats[group[counter]] = stat_group
            counter += 1

        # Gracias ChatGPT
        df = pd.DataFrame(
            {
                (outer_key, inner_key): value 
                   for outer_key, inner_dict in stats.items() 
                   for inner_key, value in inner_dict.items()
            },
            index=[nombre]
        )
        
        return df
    
    def stats_jugadores_equipo(self, team: str):
        """
        team: /id/nombre (e.g. /4608/natus-vincere)
        """
        df_jugadores = pd.DataFrame()
        dict_jugadores = self.jugadores_team(team)

        counter = 0
        while not dict_jugadores and counter < 10:
            dict_jugadores = self.jugadores_team(team)
            counter += 1
            sleep(1)

        if not dict_jugadores:
            return

        for link in dict_jugadores.values():
            jugador = link.split("?")[0][14:]
            df = self.stats_individuales(jugador)
            df_jugadores = pd.concat([df_jugadores, df])
            
        return df_jugadores

In [237]:
hltv_scraper = HltvScraper()

In [241]:
dicc = hltv_scraper.jugadores_team("/4608/natus-vincere")
dicc

{'Aleksib': '/stats/players/9816/aleksib?startDate=2023-11-07&endDate=2024-11-07&matchType=all&maps=all&rankingFilter=all',
 'iM': '/stats/players/14759/im?startDate=2023-11-07&endDate=2024-11-07&matchType=all&maps=all&rankingFilter=all',
 'b1t': '/stats/players/18987/b1t?startDate=2023-11-07&endDate=2024-11-07&matchType=all&maps=all&rankingFilter=all',
 'jL': '/stats/players/19206/jl?startDate=2023-11-07&endDate=2024-11-07&matchType=all&maps=all&rankingFilter=all',
 'w0nderful': '/stats/players/20127/w0nderful?startDate=2023-11-07&endDate=2024-11-07&matchType=all&maps=all&rankingFilter=all'}

In [242]:
df1 = hltv_scraper.stats_individuales("/9816/aleksib")
df1

Overall stats                                                     \
                Kills Deaths Kill / Death Kill / Round Rounds with kills   
aleksib          1985   2407         0.82         0.53              1503   

                                      Opening stats                       \
        Kill - Death difference Total opening kills Total opening deaths   
aleksib                    -422                 287                  377   

                                                ...   Round stats  \
        Opening kill ratio Opening kill rating  ... 2 kill rounds   
aleksib               0.76                0.88  ...           326   

                                                  Weapon stats               \
        3 kill rounds 4 kill rounds 5 kill rounds  Rifle kills Sniper kills   
aleksib            67             6             1         1324           19   

                                              
        SMG kills Pistol kills Grenade Other  
aleksib       259          320      64    20  

[1 rows x 24 columns]

In [243]:
df_jugadores_navi = hltv_scraper.stats_jugadores_equipo("/4608/natus-vincere")
df_jugadores_navi

Overall stats                                                     \
                  Kills Deaths Kill / Death Kill / Round Rounds with kills   
aleksib            1985   2407         0.82         0.53              1503   
im                 2611   2501         1.04         0.70              1761   
b1t                2718   2426         1.12         0.73              1846   
jl                 2670   2425         1.10         0.71              1792   
w0nderful          2701   2172         1.24         0.72              1849   

                                        Opening stats                       \
          Kill - Death difference Total opening kills Total opening deaths   
aleksib                      -422                 287                  377   
im                            110                 460                  434   
b1t                           292                 436                  359   
jl                            245                 412                  358   
w0nderful                     529                 386                  229   

                                                  ...   Round stats  \
          Opening kill ratio Opening kill rating  ... 2 kill rounds   
aleksib                 0.76                0.88  ...           326   
im                      1.06                1.09  ...           491   
b1t                     1.21                1.09  ...           509   
jl                      1.15                1.06  ...           484   
w0nderful               1.69                1.08  ...           413   

                                                    Weapon stats               \
          3 kill rounds 4 kill rounds 5 kill rounds  Rifle kills Sniper kills   
aleksib              67             6             1         1324           19   
im                  140            25             1         2087           13   
b1t                 142            25             1         2037           22   
jl                  154            26             2         2037           19   
w0nderful           168            29             4         1080         1100   

                                                
          SMG kills Pistol kills Grenade Other  
aleksib         259          320      64    20  
im               72          420      20    14  
b1t             198          440      23    12  
jl              142          441      27    25  
w0nderful        40          465      21    15  

[5 rows x 24 columns]